In [44]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Only sending allowed
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0)  # this opens a browser on your local machine

# Save the generated tokens for later use
with open('token.json', 'w') as token_file:
    token_file.write(creds.to_json())

# Build the Gmail service
service = build('gmail', 'v1', credentials=creds)
print("Gmail API ready!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=478084782760-jntj3gmgbsnrpo6ag1cabi2anmginqf6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54614%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=gNyBTEGNCv5rHPwlABiLgSn65jfZYJ&access_type=offline
✅ Gmail API ready!


In [ ]:
# Make sure there is a browser that is open and available to use gmail with
# This will open a new tab on that browser and prompt you to log in.
# Make sure you've downloaded the JSON file with your client secret, client id, etc. and rename it to credentials.json.
# Place it in the same directory as the cloned repo.
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.send']
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)
print("Gmail API ready!")

✅ Gmail API ready!


In [ ]:
# Send function
import base64
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

def send_email(service, to, subject, body, image_path, attach_qr=False):
    message = MIMEMultipart()
    message['to'] = to
    message['subject'] = subject

    # Attach the image
    if attach_qr:
        with open(f'{image_path}', 'rb') as img_file:
            img_data = img_file.read()
            image = MIMEImage(img_data, name='registration_code.jpg') # Adjust name and MIME type
            message.attach(image)
    
    html_message = f"""
    <html>
    <body>
    <p>{body}</p>
    </body>
    </html>
    """
    message.attach(MIMEText(html_message, _subtype="html"))
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    message = {'raw': raw}

    try:
        sent = service.users().messages().send(userId="me", body=message).execute()
        print(f"Email sent to {to} (Message ID: {sent['id']})")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [60]:
# Driver code
import os
import json
import time

root, _, curdir = next(os.walk("./qrcodes_test"))

with open("email_content.json", "r") as f:
    content = json.load(f)
    subject = content["subject"]
    body = " ".join(content["body"])
    attach_qr = content["attach_qr"]

for filename in curdir:
    # works for files that end with .jpeg, .png, etc.
    # Will not work if the file extension has multiple periods (ex. .modules.json, .abc.def)
    email = ".".join(filename.split(".")[0:-1])
    send_email(service, email, subject, body, os.path.join(root, filename), attach_qr=attach_qr)
    # Pause program for 2 seconds to combat potential rate limits
    time.sleep(2)

Email sent to datericboi@gmail.com (Message ID: 19a51316d8bee7dc)
Email sent to eldar.isgandarov@mail.mcgill.ca (Message ID: 19a51317937ba077)
Email sent to wentaoguanmcgill@gmail.com (Message ID: 19a513183c2759d0)
